<a href="https://colab.research.google.com/github/yaolihua081/AIG_PC/blob/main/Generate_False_Statement_from_a_True_sentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generate_True_or_False_OpenAI_GPT2_Sentence_BERT
#Generate True or False questions from any content with OpenAI GPT2 text generation, Sentence-BERT semantic search and Berkley Neural parser (Constituency Parsing with a Self-Attentive Encoder).

Medium Blog Post expaining the algorithm : https://medium.com/swlh/practical-ai-automatically-generate-true-or-false-questions-from-any-content-with-openai-gpt2-9081ffe4d4c9

In [1]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.3 MB 16 kB/s 
     |████████████████████████████████| 50 kB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 26.4 MB/s 
     |████████████████████████████████| 488 kB 50.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you

In [2]:
!pip install torch==1.4.0
!pip install sentence-transformers==0.2.5.1
!pip install transformers==2.6.0
!pip install benepar==0.1.2
!pip install summa
!pip install nltk==3.4.5
!pip install spacy==2.1.0
!python3 -m spacy download en
!pip install scipy

     |████████████████████████████████| 753.4 MB 6.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 52 kB 547 kB/s 
     |████████████████████████████████| 447 kB 9.4 MB/s 
     |████████████████████████████████| 1.2 MB 11.4 MB/s 
     |████████████████████████████████| 895 kB 28.8 MB/s 
     |████████████████████████████████| 131 kB 42.5 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 7.7 MB 35.0 MB/s 
     |█████████

In [3]:
import torch
import transformers
print (torch.__version__)
print (transformers.__version__)
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.4.0
2.6.0
1.14.0


In [4]:
import requests
import json
from summa.summarizer import summarize
import benepar
import string
import nltk
from nltk import tokenize
from nltk.tokenize import sent_tokenize
import re
from random import shuffle
import spacy
nlp = spacy.load('en')
#this package is required for the summa summarizer
nltk.download('punkt')
benepar.download('benepar_en2')
benepar_parser = benepar.Parser("benepar_en2")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package benepar_en2 to /root/nltk_data...


#Split sentence at appropriate place with Berkley Constituency parser.
#Use Berkley Constituency parser to split a sentence at ending verb phrase or noun phrase. Eg : If the input sentence is "Divergent plate boundaries also occur in the continental crust" we split it at the ending noun phrase to get "Divergent plate boundaries also occur in". Now we give the partial sentence "Divergent plate boundaries also occur in" to OpenAI GPT-2 to generate sentences with different endings.

In [11]:
text='''
There is a lot of volcanic activity at divergent plate boundaries in the oceans. 
For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. 
This is a divergent plate boundary that runs north-south through the middle of 
the Atlantic Ocean. As tectonic plates pull away from each other at a divergent 
plate boundary, they create deep fissures, or cracks, in the crust. 
Molten rock, called magma, erupts through these cracks onto Earth’s surface. 
At the surface, the molten rock is called lava. It cools and hardens, 
forming rock. Divergent plate boundaries also occur in the continental crust. 
Volcanoes form at these boundaries, but less often than in ocean crust. 
That’s because continental crust is thicker than oceanic crust. 
This makes it more difficult for molten rock to push up through the crust. 
Many volcanoes form along convergent plate boundaries where one tectonic 
plate is pulled down beneath another at a subduction zone. 
The leading edge of the plate melts as it is pulled into the mantle, 
forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, 
they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. 
This is why the Pacific rim is called the “Pacific Ring of Fire.”
'''

In [12]:
from string import punctuation

def preprocess(sentences):
    output = []
    for sent in sentences:
        single_quotes_present = len(re.findall(r"['][\w\s.:;,!?\\-]+[']",sent))>0
        double_quotes_present = len(re.findall(r'["][\w\s.:;,!?\\-]+["]',sent))>0
        question_present = "?" in sent
        if single_quotes_present or double_quotes_present or question_present :
            continue
        else:
            output.append(sent.strip(punctuation))
    return output
        
        
def get_candidate_sents(resolved_text, ratio=0.3):
    candidate_sents = summarize(resolved_text, ratio=ratio)
    candidate_sents_list = tokenize.sent_tokenize(candidate_sents)
    candidate_sents_list = [re.split(r'[:;]+',x)[0] for x in candidate_sents_list ]
    # Remove very short sentences less than 30 characters and long sentences greater than 150 characters
    filtered_list_short_sentences = [sent for sent in candidate_sents_list if len(sent)>30 and len(sent)<150]
    return filtered_list_short_sentences

cand_sents = get_candidate_sents(text)
filter_quotes_and_questions = preprocess(cand_sents)
for each_sentence in filter_quotes_and_questions:
    print (each_sentence)
    print ("\n")

There is a lot of volcanic activity at divergent plate boundaries in the oceans


plate boundary, they create deep fissures, or cracks, in the crust


Divergent plate boundaries also occur in the continental crust


Volcanoes form at these boundaries, but less often than in ocean crust




In [13]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final
    
def get_termination_portion(main_string,sub_string):
    combined_sub_string = sub_string.replace(" ","")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ","")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])
                     
    return None
    
def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return get_flattened(last_NP),get_flattened(last_VP)
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
        
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


def get_sentence_completions(key_sentences):
    sentence_completion_dict = {}
    for individual_sentence in filter_quotes_and_questions:
        sentence = individual_sentence.rstrip('?:!.,;')
        tree = benepar_parser.parse(sentence)
        last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
        phrases= []
        if last_verbphrase is not None:
            verbphrase_string = get_termination_portion(sentence,last_verbphrase)
            phrases.append(verbphrase_string)
        if last_nounphrase is not None:
            nounphrase_string = get_termination_portion(sentence,last_nounphrase)
            phrases.append(nounphrase_string)

        longest_phrase =  sorted(phrases, key=len,reverse= True)
        if len(longest_phrase) == 2:
            first_sent_len = len(longest_phrase[0].split())
            second_sentence_len = len(longest_phrase[1].split())
            if (first_sent_len - second_sentence_len) > 4:
                del longest_phrase[1]
                
        if len(longest_phrase)>0:
            sentence_completion_dict[sentence]=longest_phrase
    return sentence_completion_dict



sent_completion_dict = get_sentence_completions(filter_quotes_and_questions)

print (sent_completion_dict)    

{'There is a lot of volcanic activity at divergent plate boundaries in the oceans': ['There is a lot of volcanic activity at divergent plate boundaries in'], 'plate boundary, they create deep fissures, or cracks, in the crust': ['plate boundary, they create deep fissures, or cracks, in'], 'Divergent plate boundaries also occur in the continental crust': ['Divergent plate boundaries also occur in', 'Divergent plate boundaries also'], 'Volcanoes form at these boundaries, but less often than in ocean crust': ['Volcanoes form at these boundaries, but less often than in']}


#Load OpenAI GPT2 and Sentence BERT.
1.Use huggingface transformers library to generate text with OpenAI GPT-2 and sentence BERT for filtering similar generated sentences.

In [14]:
# https://huggingface.co/transformers/main_classes/model.html?highlight=no_repeat_ngram_size

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)

from sentence_transformers import SentenceTransformer
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md
model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:25<00:00, 15.8MB/s]


#Filter sentences and generate false sentences.
1.Generate multiple sentences (OpenAI GPT2) and among them filter (Sentence BERT) the ones that are similar, since we want to keep only dissimilar ones as False sentences.

In [15]:
from nltk import tokenize
import scipy
torch.manual_seed(2020)


def sort_by_similarity(original_sentence,generated_sentences_list):
    # Each sentence is encoded as a 1-D vector with 768 columns
    sentence_embeddings = model_BERT.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = model_BERT.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            if score < 0.9:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
           
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    
    return sorted_dissimilar_sentences[:3]
            
def generate_sentences(partial_sentence,full_sentence):
    input_ids = torch.tensor([tokenizer.encode(partial_sentence)])
    maximum_length = len(partial_sentence.split())+80

    # Actiavte top_k sampling and top_p sampling with only from 90% most likely words
    sample_outputs = model.generate(
        input_ids, 
        do_sample=True, 
        max_length=maximum_length, 
        top_p=0.90, # 0.85 
        top_k=50,   #0.30
        repetition_penalty  = 10.0,
        num_return_sequences=10
    )
    generated_sentences=[]
    for i, sample_output in enumerate(sample_outputs):
        decoded_sentences = tokenizer.decode(sample_output, skip_special_tokens=True)
        decoded_sentences_list = tokenize.sent_tokenize(decoded_sentences)
        generated_sentences.append(decoded_sentences_list[0])
        
    top_3_sentences = sort_by_similarity(full_sentence,generated_sentences)
    
    return top_3_sentences

index = 1
choice_list = ["a)","b)","c)","d)","e)","f)"]
for key_sentence in sent_completion_dict:
    partial_sentences = sent_completion_dict[key_sentence]
    false_sentences =[]
    print_string = "**%s) True Sentence (from the story) :**"%(str(index))
    printmd(print_string)
    print ("  ",key_sentence)
    for partial_sent in partial_sentences:
        false_sents = generate_sentences(partial_sent,key_sentence)
        false_sentences.extend(false_sents)
    printmd("  **False Sentences (GPT-2 Generated)**")
    for ind,false_sent in enumerate(false_sentences):
        print_string_choices = "**%s** %s"%(choice_list[ind],false_sent)
        printmd(print_string_choices)
    index = index+1
    
    print ("\n\n")


**1) True Sentence (from the story) :**

   There is a lot of volcanic activity at divergent plate boundaries in the oceans


  **False Sentences (GPT-2 Generated)**

**a)** There is a lot of volcanic activity at divergent plate boundaries in Africa.

**b)** There is a lot of volcanic activity at divergent plate boundaries in Greenland," he says.

...

**e)** Divergent plate boundaries also affect the level of activity.

**f)** Divergent plate boundaries also prevent water pollution, as well.

**4) True Sentence (from the story) :**

   Volcanoes form at these boundaries, but less often than in ocean crust


  **False Sentences (GPT-2 Generated)**